In [ ]:
from pandas import Series
from matplotlib import pyplot
from pandas import DataFrame
from pandas import set_option
from pandas import concat
import numpy as np
import math
from sklearn.linear_model import LinearRegression

Изначально планировалось использование пакеты statsmodels и функции 
statsmodels.tsa.seasonal.seasonal_decompose(), однако результат был крайне странный, скорее всего из-за небольшого размера выборки. Поэтому пришлось раскладывать ряд "вручную".


In [ ]:
#get data
data = [15.65, 10.56, 8.62, 14.41, 16.93, 11.12,
                  7.58, 16.23, 17.38, 13.89, 8.73, 16.77, 18.49, 10.73,
                  9.89, 17.57, 20.56, 14.14, 8.19, 18.94]
index = []
for i in range (1,21,1):
  index.append(i)

In [ ]:
#prepare data by rolling mean method
series = Series(data,index)
rolling = series.rolling(window=4)
rolling_mean = rolling.mean()
data_table = concat([series, rolling_mean], axis =1)
data_table.columns = ['y', 'y~']
print(data_table)

        y       y~
1   15.65      NaN
2   10.56      NaN
3    8.62      NaN
4   14.41  12.3100
5   16.93  12.6300
6   11.12  12.7700
7    7.58  12.5100
8   16.23  12.9650
9   17.38  13.0775
10  13.89  13.7700
11   8.73  14.0575
12  16.77  14.1925
13  18.49  14.4700
14  10.73  13.6800
15   9.89  13.9700
16  17.57  14.1700
17  20.56  14.6875
18  14.14  15.5400
19   8.19  15.1150
20  18.94  15.4575


In [ ]:
# get seasonal y/y~
seasonal_y = []
for i in data_table.index:
  if math.isnan(data_table['y~'][i]):
    seasonal_y.append(np.nan)
  else:
    seasonal_y.append(data_table['y'][i] / data_table['y~'][i])

seasonal_y = Series(seasonal_y,index)
data_table = concat([series, rolling_mean, seasonal_y], axis =1)
data_table.columns = ['y', 'y~','y/y~']
print(data_table)



  


        y       y~      y/y~
1   15.65      NaN       NaN
2   10.56      NaN       NaN
3    8.62      NaN       NaN
4   14.41  12.3100  1.170593
5   16.93  12.6300  1.340459
6   11.12  12.7700  0.870791
7    7.58  12.5100  0.605915
8   16.23  12.9650  1.251832
9   17.38  13.0775  1.329000
10  13.89  13.7700  1.008715
11   8.73  14.0575  0.621021
12  16.77  14.1925  1.181610
13  18.49  14.4700  1.277816
14  10.73  13.6800  0.784357
15   9.89  13.9700  0.707946
16  17.57  14.1700  1.239944
17  20.56  14.6875  1.399830
18  14.14  15.5400  0.909910
19   8.19  15.1150  0.541846
20  18.94  15.4575  1.225295


In [ ]:
#split data to quaters
first = []
second = []
third = []
forth = []
for i in range(1,18,4):
  first.append(data_table['y/y~'][i])
  second.append(data_table['y/y~'][i + 1])
  third.append(data_table['y/y~'][i + 2])
  forth.append(data_table['y/y~'][i + 3])

print('1st quater ',first)
print('2nd quater ', second)
print('3rd quater ', third)
print('4th quater ', forth)

1st quater  [nan, 1.3404592240696755, 1.3290001911680371, 1.2778161713890812, 1.3998297872340424]
2nd quater  [nan, 0.870790916209867, 1.0087145969498912, 0.784356725146199, 0.9099099099099099]
3rd quater  [nan, 0.6059152677857715, 0.6210208073981862, 0.7079455977093775, 0.5418458484948727]
4th quater  [1.1705930138099108, 1.2518318549942156, 1.1816100052844816, 1.2399435426958365, 1.2252951641597933]


In [ ]:
sum_first = 0
for i in first:
  if math.isnan(i) is False:
    sum_first = sum_first + i
mean_first = sum_first / 4
print('summary of 1st quaters', sum_first)

sum_second = 0
for i in second:
  if math.isnan(i) is False:
    sum_second = sum_second + i
mean_second = sum_second / 4
print('summary of 2nd quaters', sum_second)

sum_third = 0
for i in third:
  if math.isnan(i) is False:
    sum_third = sum_third + i
mean_third = sum_third / 4
print('summary of 3rd quaters', sum_third)


sum_forth = 0
for i in forth:
  if math.isnan(i) is False:
    sum_forth = sum_forth + i
mean_forth = sum_forth / 5
print('summary of 4th quaters', sum_forth)
print()



print('mean of 1st quaters', mean_first)
print('mean of 2nd quaters', mean_second)
print('mean of 3nd quaters', mean_third)
print('mean of 4th quaters', mean_forth)
print()

summary_means = mean_first + mean_second + mean_third + mean_forth
corrected = 4 / summary_means

mean_first = mean_first * corrected
mean_second = mean_second * corrected
mean_third = mean_third * corrected
mean_forth = mean_forth * corrected
print('corrected indexes:', mean_first, '',mean_second,'',mean_third,'',mean_forth)




summary of 1st quaters 5.3471053738608365
summary of 2nd quaters 3.573772148215867
summary of 3rd quaters 2.4767275213882076
summary of 4th quaters 6.069273580944238

mean of 1st quaters 1.3367763434652091
mean of 2nd quaters 0.8934430370539668
mean of 3nd quaters 0.6191818803470519
mean of 4th quaters 1.2138547161888478

corrected indexes: 1.3159656699099365  0.8795340899999188  0.6095425775225868  1.1949576625675578


In [ ]:
# get s(t)
s = []
for i in range(1,18,4):
  s.append(mean_first)
  s.append(mean_second)
  s.append(mean_third)
  s.append(mean_forth)

data_table['s'] = Series(s,index)
print(data_table)

        y       y~      y/y~         s
1   15.65      NaN       NaN  1.315966
2   10.56      NaN       NaN  0.879534
3    8.62      NaN       NaN  0.609543
4   14.41  12.3100  1.170593  1.194958
5   16.93  12.6300  1.340459  1.315966
6   11.12  12.7700  0.870791  0.879534
7    7.58  12.5100  0.605915  0.609543
8   16.23  12.9650  1.251832  1.194958
9   17.38  13.0775  1.329000  1.315966
10  13.89  13.7700  1.008715  0.879534
11   8.73  14.0575  0.621021  0.609543
12  16.77  14.1925  1.181610  1.194958
13  18.49  14.4700  1.277816  1.315966
14  10.73  13.6800  0.784357  0.879534
15   9.89  13.9700  0.707946  0.609543
16  17.57  14.1700  1.239944  1.194958
17  20.56  14.6875  1.399830  1.315966
18  14.14  15.5400  0.909910  0.879534
19   8.19  15.1150  0.541846  0.609543
20  18.94  15.4575  1.225295  1.194958


In [ ]:
#get d(t):
d= []
for i in data_table.index:
  d.append(data_table['y'][i] / data_table['s'][i])

data_table['d'] = Series(d,index)
print(data_table)

        y       y~      y/y~         s          d
1   15.65      NaN       NaN  1.315966  11.892407
2   10.56      NaN       NaN  0.879534  12.006357
3    8.62      NaN       NaN  0.609543  14.141752
4   14.41  12.3100  1.170593  1.194958  12.059005
5   16.93  12.6300  1.340459  1.315966  12.865077
6   11.12  12.7700  0.870791  0.879534  12.643057
7    7.58  12.5100  0.605915  0.609543  12.435555
8   16.23  12.9650  1.251832  1.194958  13.582071
9   17.38  13.0775  1.329000  1.315966  13.207031
10  13.89  13.7700  1.008715  0.879534  15.792452
11   8.73  14.0575  0.621021  0.609543  14.322215
12  16.77  14.1925  1.181610  1.194958  14.033970
13  18.49  14.4700  1.277816  1.315966  14.050519
14  10.73  13.6800  0.784357  0.879534  12.199641
15   9.89  13.9700  0.707946  0.609543  16.225282
16  17.57  14.1700  1.239944  1.194958  14.703450
17  20.56  14.6875  1.399830  1.315966  15.623508
18  14.14  15.5400  0.909910  0.879534  16.076694
19   8.19  15.1150  0.541846  0.609543  13.436305


In [ ]:
#get u(t) by linear regression, u(t) = a + bt
x = np.array(index).reshape((-1, 1))
y = np.array(data_table['d'])
model = LinearRegression().fit(x,y)
a = model.intercept_
b = model.coef_[0]
print('a: ',a,'b: ',b)
u = model.predict(x)

data_table['u'] = Series(u,index)
print(data_table)


a:  12.04919572165011 b:  0.17220174681444297
        y       y~      y/y~         s          d          u
1   15.65      NaN       NaN  1.315966  11.892407  12.221397
2   10.56      NaN       NaN  0.879534  12.006357  12.393599
3    8.62      NaN       NaN  0.609543  14.141752  12.565801
4   14.41  12.3100  1.170593  1.194958  12.059005  12.738003
5   16.93  12.6300  1.340459  1.315966  12.865077  12.910204
6   11.12  12.7700  0.870791  0.879534  12.643057  13.082406
7    7.58  12.5100  0.605915  0.609543  12.435555  13.254608
8   16.23  12.9650  1.251832  1.194958  13.582071  13.426810
9   17.38  13.0775  1.329000  1.315966  13.207031  13.599011
10  13.89  13.7700  1.008715  0.879534  15.792452  13.771213
11   8.73  14.0575  0.621021  0.609543  14.322215  13.943415
12  16.77  14.1925  1.181610  1.194958  14.033970  14.115617
13  18.49  14.4700  1.277816  1.315966  14.050519  14.287818
14  10.73  13.6800  0.784357  0.879534  12.199641  14.460020
15   9.89  13.9700  0.707946  0.609543 

In [ ]:
#get d(t)/u(t) = c(t)e(t)
ce= []
for i in data_table.index:
  ce.append(data_table['d'][i] / data_table['u'][i])

data_table['d/u'] = Series(ce,index)
set_option("display.max_rows", None, "display.max_columns", None)
print('Print just last 3 columns, because all table is printed in unreadable way')
print(data_table[['d', 'u', 'd/u']])


Print just last 3 columns, because all table is printed in unreadable way
            d          u       d/u
1   11.892407  12.221397  0.973081
2   12.006357  12.393599  0.968755
3   14.141752  12.565801  1.125416
4   12.059005  12.738003  0.946695
5   12.865077  12.910204  0.996505
6   12.643057  13.082406  0.966417
7   12.435555  13.254608  0.938206
8   13.582071  13.426810  1.011564
9   13.207031  13.599011  0.971176
10  15.792452  13.771213  1.146773
11  14.322215  13.943415  1.027167
12  14.033970  14.115617  0.994216
13  14.050519  14.287818  0.983391
14  12.199641  14.460020  0.843681
15  16.225282  14.632222  1.108873
16  14.703450  14.804424  0.993179
17  15.623508  14.976625  1.043193
18  16.076694  15.148827  1.061250
19  13.436305  15.321029  0.876985
20  15.849934  15.493231  1.023023


In [ ]:
# get c(t) by rolling mean method
c = Series(data_table['d/u'],index)
c_rolling = c.rolling(window = 3)
c = c_rolling.mean()
data_table['c'] = Series(c,index)
print(data_table[['d', 'u', 'd/u','c']])

            d          u       d/u         c
1   11.892407  12.221397  0.973081       NaN
2   12.006357  12.393599  0.968755       NaN
3   14.141752  12.565801  1.125416  1.022417
4   12.059005  12.738003  0.946695  1.013622
5   12.865077  12.910204  0.996505  1.022872
6   12.643057  13.082406  0.966417  0.969872
7   12.435555  13.254608  0.938206  0.967042
8   13.582071  13.426810  1.011564  0.972062
9   13.207031  13.599011  0.971176  0.973649
10  15.792452  13.771213  1.146773  1.043171
11  14.322215  13.943415  1.027167  1.048372
12  14.033970  14.115617  0.994216  1.056052
13  14.050519  14.287818  0.983391  1.001591
14  12.199641  14.460020  0.843681  0.940429
15  16.225282  14.632222  1.108873  0.978649
16  14.703450  14.804424  0.993179  0.981911
17  15.623508  14.976625  1.043193  1.048415
18  16.076694  15.148827  1.061250  1.032541
19  13.436305  15.321029  0.876985  0.993809
20  15.849934  15.493231  1.023023  0.987086


In [ ]:
# get e(t) = d(t) / (u(t)*c(t))
e= []
for i in data_table.index:
  if math.isnan(data_table['c'][i]) is False:
    e.append(data_table['d'][i] / (data_table['u'][i] * data_table['c'][i]))
  else:
    e.append(np.nan)

data_table['e'] = Series(e,index)
print('Final table')
print(data_table[['y','y~','y/y~','s', 'd', 'u', 'd/u','c', 'e']])


Final table
        y       y~      y/y~         s          d          u       d/u  \
1   15.65      NaN       NaN  1.315966  11.892407  12.221397  0.973081   
2   10.56      NaN       NaN  0.879534  12.006357  12.393599  0.968755   
3    8.62      NaN       NaN  0.609543  14.141752  12.565801  1.125416   
4   14.41  12.3100  1.170593  1.194958  12.059005  12.738003  0.946695   
5   16.93  12.6300  1.340459  1.315966  12.865077  12.910204  0.996505   
6   11.12  12.7700  0.870791  0.879534  12.643057  13.082406  0.966417   
7    7.58  12.5100  0.605915  0.609543  12.435555  13.254608  0.938206   
8   16.23  12.9650  1.251832  1.194958  13.582071  13.426810  1.011564   
9   17.38  13.0775  1.329000  1.315966  13.207031  13.599011  0.971176   
10  13.89  13.7700  1.008715  0.879534  15.792452  13.771213  1.146773   
11   8.73  14.0575  0.621021  0.609543  14.322215  13.943415  1.027167   
12  16.77  14.1925  1.181610  1.194958  14.033970  14.115617  0.994216   
13  18.49  14.4700  1.2778